<a href="https://colab.research.google.com/github/agcorreas/KNN_TP/blob/main/TP_2_Metodos_Numericos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP 2 - Métodos Numéricos
## Grupo 11

### Introducción
Se recomienda ejecutar el notebook en el entorno de Google Colab para poder replicar los resultados obtenidos.

<!--El notebook está ordenado por consigna por lo que pueden haber bloques de código dependiendo de alguno definido anteriormente. Se recomienda ejecutar todo el notebook la primera vez y luego ir descomentando los métodos de test para ir probando cada punto.-->

El notebook está divido entre las secciones de la consigna:
- KNN
- Método de la potencia
- Convergencia del Método de la potencia

### KNN

La sección de KNN tiene la implementación del algoritmo de K-Nearest Neighbors y la función `exactitud(k, X_train, y_train, X_dev, y_dev)` que dados los datos de los `k` vecinos a utilizar, y los conjuntos de datos de entrenaentramientomiento (`X_train, y_train`) y de prueba (`X_dev, y_dev`) calcula el porcentaje de exactitud de la implementación.

### Método de la potencia

La sección del Método de la potencia contiene la busqueda de los hiper parametros que maximizan el porcetaje de exactitud para una partición de los datos de entrenamiento en la función `cross_validation_autovectores()`. También se incluye la comparación contra el conjunto total de entrenamiento.
La función `PCA(C)` toma una matriz `C` y calcula sus autovectores utilizando el método de la potencia programado utilizando `eigen`.

### Convergencia del Método de la potencia

En esta sección se calcularemos el resultado de aplicar método de la potencia con matrices Householder aleatorias y el error cometido en base a un parametro _epsilon_.
La función `test_error()` genera los datos necesarios para graficar el promedio del error y el desvio estandar del mismo como también para el gráfico de cantidad de iteraciones necesarias para generar los autovalores utilizando el método de la potencia para cada _epsilon_.

---

In [ ]:
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
from numpy import dot
from numpy.linalg import norm

## KNN

In [ ]:
!wget -O datos.zip https://www.dropbox.com/scl/fi/v6qfj1ktarocr8sl02r8k/datos.zip?rlkey=2u060s5619gvcvnnnhq93rn4e&st=jy3dah88&dl=1
!unzip datos.zip

--2025-07-22 04:51:03--  https://www.dropbox.com/scl/fi/v6qfj1ktarocr8sl02r8k/datos.zip?rlkey=2u060s5619gvcvnnnhq93rn4e
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc72a7a9e51f64f8642210756096.dl.dropboxusercontent.com/cd/0/inline/Ct--tTYR_06looClybB8aN1YiL2my7LCMd8EMh_UZ1q5LMdRAo9NGqvDoPhDwvn4yP8THfTx9wucG7_KJqYXLiVX0qp-G6l6XJspGiVvAv2MvAjuGbNOwLJJJCEvYH-Z2sLPA7Fb3Fw2aLEtE5JmYHy3/file# [following]
--2025-07-22 04:51:04--  https://uc72a7a9e51f64f8642210756096.dl.dropboxusercontent.com/cd/0/inline/Ct--tTYR_06looClybB8aN1YiL2my7LCMd8EMh_UZ1q5LMdRAo9NGqvDoPhDwvn4yP8THfTx9wucG7_KJqYXLiVX0qp-G6l6XJspGiVvAv2MvAjuGbNOwLJJJCEvYH-Z2sLPA7Fb3Fw2aLEtE5JmYHy3/file
Resolving uc72a7a9e51f64f8642210756096.dl.dropboxusercontent.com (uc72a7a9e51f64f8642210756096.dl.dropboxusercontent.com)... 162.125.3.15, 2620:100:

In [ ]:


class_names = ["T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

X_train = np.loadtxt("datos/X_train.csv", delimiter=",")
y_train = np.loadtxt("datos/y_train.csv", delimiter=",").astype(int)
X_test = np.loadtxt("datos/X_test.csv", delimiter=",")
y_test = np.loadtxt("datos/y_test.csv", delimiter=",").astype(int)

X_train.shape, y_train.shape, X_test.shape, y_test.shape



((5000, 784), (5000,), (500, 784), (500,))

In [ ]:
import copy

def distancia(X):
  Y = copy.copy(X)
  media_por_columna = np.mean(Y,axis = 1) #! Esto calcula la media por fila
  Y = Y - media_por_columna.reshape((Y.shape[0],1))
  Y_norma = np.linalg.norm(Y,axis=1,keepdims=True)
  Y = Y / Y_norma
  Z = Y @ Y.T
  # unos = np.full((len(Z),len(Z)),1)
  dist = 1 - Z
  # print(dist.shape)
  return dist

#print(distancia(X_train))


k = 5

#! Traten de no dejar codigo y celdas enteras comentadas
# x = prenda que quiero clasificar
# X_train = base de datos de prendas
# y_train = clasificaciones de las prendas de X_train
# def KNN(k,x,X_train,y_train):

#     #print(matriz_distancia.shape)
#     distancias = {'index': [], 'dist': []}
#     for i in range(0,len(X_train)-1):
#       distancias['index'].append(i)
#       distancias['dist'].append(matriz_distancia[i][len(matriz_distancia)-1])

#     # Obtengo un array con los k índices (en X_train) de las menores distancias

#     k_vecinos_cercanos = np.argsort(distancias['dist'])[0:k-1]#,order=['dist','index'])[0:k-1]
#     # k_min_distancias = np.array(distancias['dist'])[k_min_distancias]
#     # Indexo y_train en estos k índices para obtener qué tipo de prenda son los k vecinos más cercanos
#     prendas_cercanas = []
#     for i in k_vecinos_cercanos:
#       prendas_cercanas.append(y_train[i])
#     # Veo cuál tipo de prenda se repite más en los k vecinos más cercanos
#     prediction = st.mode(prendas_cercanas)[0]
#     return prediction

def KNN(k,distancias_a_x,y_train):
  # indices = np.arange(0,len(distancias_a_x)-1)
  # distancias = {'index': indices, 'dist': distancias_a_x}
  # Obtengo un array con los k índices en X_train de las menores distancias
  #k_vecinos_cercanos = np.argsort(distancias['dist'])[0:k-1]
  k_vecinos_cercanos = np.argsort(distancias_a_x)[0:k-1]
  prendas_cercanas = []
  for i in k_vecinos_cercanos: #! Numpy permite indexar con lista de indices, se pueden ahorrar este for
    prendas_cercanas.append(y_train[i])
  # Veo cuál tipo de prenda se repite más en los k vecinos más cercanos
  prediction = st.mode(prendas_cercanas)[0]
  return prediction

def exactitud(k,X_train,y_train,X_dev,y_dev):
  # Creo la matriz de distancias
  X_train_tmp = np.concatenate((X_train,X_dev),axis=0)
  # Me quedo solo con la parte de la matriz que compara los elementos de X_dev con los de X_train
  matriz_distancia = distancia(X_train_tmp)[:len(X_dev)-1,len(X_dev):] #! Cuidado que se estan quedando un pedazo incorrecto de la matriz como distancias


  aciertos = []
  for i in range(len(X_dev)-1): #! Traten de eliminar este for
    prediction = KNN(k,matriz_distancia[i],y_train)
    # print(prediction)

    if prediction == y_dev[i]:
      aciertos.append(1)
    else:
      aciertos.append(0)
  return np.mean(aciertos) # Devuelvo el promedio del vector de aciertos

print(exactitud(k,X_train,y_train,X_test,y_test))

0.8196392785571143


## Metodo de la potencia


Para poder ejecutar el método de la potencia más eficientemente se realiza una implementación en C++ utilizando la libreria `eigen`.
Se instalan las dependencias necesarias,  se implementa el algoritmo y se compila utilizando `g++` con el flag `-O3`.

In [ ]:
# !git clone https://gitlab.com/libeigen/eigen.git
!apt-get install libeigen3-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libeigen3-dev is already the newest version (3.4.0-2ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
%%file eigen_test.cpp





#include <iostream>
#include <eigen3/Eigen/Dense>
#include <utility>
#include <vector>
#include <fstream>
#include <eigen3/Eigen/Dense>

using Eigen::MatrixXf;
using Eigen::VectorXf;

using namespace std;

pair<Eigen::VectorXf,float> metpot(const Eigen::MatrixXf& B, Eigen::VectorXf x_0, int niter, float epsilon){
    double res = 0;
    int j = 1;
    Eigen::VectorXf v = std::move(x_0); //#! Generenlo al azar aca mismo en vez de pasarlo como parametro
    Eigen::VectorXf vant = v;
    Eigen::RowVectorXf vtrans = v.transpose();
    for(int i = 0; i<niter; i++){
        v = B*v;
        v.normalize();
        vtrans = v.transpose(); //#!  No es necesario calcularlo en todas las iteraciones
        res = (vtrans*B)*v; //#! No es necesario calcularlo en todas las iteraciones
        j++;
        if((v - vant).lpNorm<Eigen::Infinity>() < epsilon){
            break;
        }
        vant = v;
    }
    //cout<<v;
    //printf("pare en la iter %d \n", j);
    return make_pair(v,res);

}

vector<pair<Eigen::VectorXf,float>> metpotdefl(Eigen::MatrixXf B, const Eigen::VectorXf& x_0, int niter, float epsilon){
    vector<pair<Eigen::VectorXf,float>> res;
    Eigen::MatrixXf C = std::move(B);
    for(int i = 0; i<C.cols(); i++){
        res.push_back(metpot(C,x_0,niter,epsilon));
        Eigen::VectorXf vnormal = res[i].first;
        Eigen::RowVectorXf vtrans = res[i].first.transpose();
        Eigen::MatrixXf vgrande = vnormal * vtrans;
        vgrande = (res[i].second*vgrande);
        C = C - vgrande;
    }
    return res;
}

int main(int argc, char** argv) {
    if (argc != 3) {
        std::cerr << "Usage: " << argv[0] << " input_file output_file" << std::endl;
        return 1;
    }

    const char* input_file = argv[1];
    const char* output_file = argv[2];

    std::ifstream fin(input_file);
    if (!fin.is_open()) {
        std::cerr << "Error: could not open input file " << input_file << std::endl;
        return 1;
    }

    // Read matrix and vector from file
    int nrows, ncols;
    fin >> nrows >> ncols;

    MatrixXf A(nrows, ncols);
    for (int i = 0; i < nrows; i++) {
        for (int j = 0; j < ncols; j++) {
            fin >> A(i, j);
        }
    }

    VectorXf b(ncols);
    for (int i = 0; i < ncols; i++) {
        fin >> b(i);
    }

    fin.close();

    b.normalize();

    // Perform matrix-vector multiplication
    vector<pair<Eigen::VectorXf,float>> c = metpotdefl(A,b,10000,1e-7);
    cout<<A<<endl;
    cout<<b<<endl;

    // Write result to output file
    std::ofstream fout(output_file);
    if (!fout.is_open()) {
        std::cerr << "Error: could not open output file " << output_file << std::endl;
        return 1;
    }

        for(int i = 0; i<c.size(); i++){
            fout<<c[i].first.transpose()<<endl;
            //fout<<c[i].second<<endl; //autovalor <- sacar para pca
        }


    fout.close();

    return 0;
}









Overwriting eigen_test.cpp


In [ ]:
%%bash

g++ -O3 eigen_test.cpp -o out


In [ ]:
#np.loadtxt('output_data.txt')

In [ ]:
#A = np.array([[1, 2, 3], [2, 5, 6], [3, 6, 9]], dtype=np.float64)
#b = np.array([1, 0, 0], dtype=np.float64)

#b = np.zeros(len(C))
#b[0] = 1
#print(C.shape)
#!rm input_data.txt
#with open('input_data.txt','a') as f:
#    f.write(f"{C.shape[0]} {C.shape[1]}\n")
#    np.savetxt(f,C, newline="\n")
#    np.savetxt(f,b.reshape(1,-1), fmt='%1.3f', newline="\n")
#!cat input_data.txt

In [ ]:
def PCA(C):
  b = np.zeros(len(C))
  b[0] = 1
  print(C.shape)
  !rm input_data.txt
  with open('input_data.txt','a') as f:
      f.write(f"{C.shape[0]} {C.shape[1]}\n")
      np.savetxt(f,C, newline="\n")
      np.savetxt(f,b.reshape(1,-1), fmt='%1.3f', newline="\n")
  #!cat input_data.txt
  !./out input_data.txt output_data.txt
  array_autovectores = np.fromfile('output_data.txt')
  array_autovectores = array_autovectores.T
  return array_autovectores
#PCA(C)

In [ ]:
X_train_np = np.array(X_train)
y_train_np = np.array(y_train)
split_X = np.array_split(X_train_np,5,axis=0)
split_Y = np.array_split(y_train_np,5)
for i in range(0, len(split_X)-1):
  X_dev_temp = split_X[i]
  y_dev_temp = split_Y[i]
  X_train_temp = np.concatenate([split_X[j] for j in range(len(split_X)) if j != i])
  y_train_temp = np.concatenate([split_Y[j] for j in range(len(split_Y)) if j != i])

In [ ]:
def cross_validation_autovectores():
  # Creo las 5 particiones de ambos arrays
  X_train_np = np.array(X_train)
  y_train_np = np.array(y_train)
  split_X = np.array_split(X_train_np,5,axis=0)
  split_Y = np.array_split(y_train_np,5)
  maxima_exactitud = 0
  optimos = (-1,-1)
  for i in range(0, len(split_X)-1):
    X_dev_temp = split_X[i]
    y_dev_temp = split_Y[i]
    X_train_temp = np.array([])
    y_train_temp = np.array([])
    X_train_temp = np.concatenate([split_X[j] for j in range(len(split_X)) if j != i])
    y_train_temp = np.concatenate([split_Y[j] for j in range(len(split_Y)) if j != i])

    # Matriz de covarianza
    # print(X.shape, X.T.shape)
    C = (X_train_temp.T @ X_train_temp) / (len(X_train_temp)-1) #!  Falta centrar, hagan funciones que calculen estas cosas
    PCA(C)
    V = np.fromfile('output_data.txt', sep=' ').reshape(784, 784).T
    for k in range(1,5):
      for p in range (1,784,25):
        X_hat_train = X_train_temp @ V[:,:p]
        X_hat_dev = X_dev_temp @ V[:,:p]
        res = exactitud(k,X_hat_train,y_train_temp,X_hat_dev,y_dev_temp)
        if res > maxima_exactitud:
          maxima_exactitud = res
          optimos = (k,p)
  return optimos

# cross_validation_autovectores()



In [ ]:
def test():
    optimos = cross_validation_autovectores()
    k = optimos[0]
    p = optimos[1]
    C = (X_train.T @ X_train) / (len(X_train)-1)
    PCA(C)

    V = np.fromfile('output_data.txt', sep=' ').reshape(784, 784).T
    X_hat_train = X_train @ V[:,:p]
    X_hat_test = X_test @ V[:,:p]
    return exactitud(k,X_hat_train,y_train,X_hat_test,y_test)

test()

(784, 784)
^C


FileNotFoundError: [Errno 2] No such file or directory: 'output_data.txt'

## Convergencia del Método de la potencia

In [ ]:
%%file eigen_test2.cpp





#include <iostream>
#include <eigen3/Eigen/Dense>
#include <utility>
#include <vector>
#include <fstream>
#include <eigen3/Eigen/Dense>
#include <tuple>

using Eigen::MatrixXf;
using Eigen::VectorXf;

using namespace std;

tuple<Eigen::VectorXf,float,int,float> metpot(const Eigen::MatrixXf& B, Eigen::VectorXf x_0, int niter, float epsilon){
    double res = 0;
    int j = 1;
    Eigen::VectorXf v = std::move(x_0);
    Eigen::VectorXf vant = v;
    Eigen::RowVectorXf vtrans = v.transpose();
    for(int i = 0; i<niter; i++){
        v = B*v;
        v.normalize();
        vtrans = v.transpose();
        res = (vtrans*B)*v;
        j++;
        if((v - vant).lpNorm<Eigen::Infinity>() < epsilon){
            break;
        }
        vant = v;
    }
    //cout<<v;
    //printf("pare en la iter %d \n", j);
    float error = (B*v - res*v).norm();
    return make_tuple(v,res,j,error);

}

vector<tuple<Eigen::VectorXf,float,int,float>> metpotdefl(Eigen::MatrixXf B, const Eigen::VectorXf& x_0, int niter, float epsilon){
    vector<tuple<Eigen::VectorXf,float,int,float>> res;
    Eigen::MatrixXf C = std::move(B);
    for(int i = 0; i<C.cols(); i++){
        res.push_back(metpot(C,x_0,niter,epsilon));
        Eigen::VectorXf vnormal = get<0>(res[i]);
        Eigen::RowVectorXf vtrans = get<0>(res[i]).transpose();
        Eigen::MatrixXf vgrande = vnormal * vtrans;
        vgrande = (get<1>(res[i])*vgrande);
        C = C - vgrande;
    }
    return res;
}

int main(int argc, char** argv) {
    if (argc != 3) {
        std::cerr << "Usage: " << argv[0] << " input_file output_file" << std::endl;
        return 1;
    }

    const char* input_file = argv[1];
    const char* output_file = argv[2];

    std::ifstream fin(input_file);
    if (!fin.is_open()) {
        std::cerr << "Error: could not open input file " << input_file << std::endl;
        return 1;
    }

    // Read matrix and vector from file
    int nrows, ncols;
    fin >> nrows >> ncols;

    MatrixXf A(nrows, ncols);
    for (int i = 0; i < nrows; i++) {
        for (int j = 0; j < ncols; j++) {
            fin >> A(i, j);
        }
    }

    VectorXf b(ncols);
    for (int i = 0; i < ncols; i++) {
        fin >> b(i);
    }

    fin.close();

    b.normalize();

    // Perform matrix-vector multiplication
    vector<tuple<Eigen::VectorXf,float,int,float>> c = metpotdefl(A,b,10000,1e-7);


    // Write result to output file
    std::ofstream fout(output_file);
    if (!fout.is_open()) {
        std::cerr << "Error: could not open output file " << output_file << std::endl;
        return 1;
    }

        for(int i = 0; i<c.size(); i++){
            fout<<get<0>(c[i]).transpose()<<endl;
            fout<<get<1>(c[i])<<endl;
            fout<<get<2>(c[i])<<endl;
            fout<<get<3>(c[i])<<endl;
        }


    fout.close();

    return 0;
}

/*int main()
{
//
//
    Eigen::VectorXf x = Eigen::VectorXf::Zero(150);

    // Asignar 1 al primer elemento
    x(0) = 1;

    x.normalize();


    Eigen::VectorXf v(150);
    v.setRandom();
    v.normalize();

    Eigen::RowVectorXf vtrans = v.transpose();


    Eigen::VectorXf diag(150);
    diag<< 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
            21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
            41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
            61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80,
            81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100,
            101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
            121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
            141, 142, 143, 144, 145, 146, 147, 148, 149, 150;
    Eigen::DiagonalMatrix<float, Eigen::Dynamic> matDiag(diag);

    Eigen::MatrixXf identityMatrix = Eigen::MatrixXf::Identity(150,150);


    Eigen::MatrixXf Q = identityMatrix - 2*v*vtrans;
    Eigen::MatrixXf Qtrans = Q.transpose();
    Eigen::MatrixXf A = Qtrans*matDiag*Q;

    vector<pair<Eigen::VectorXf,float>> res = metpotdefl(A,x,10000,1e-7);


    for(int j = 0; j<150; j++){
        printf("inicio eigen %d \n",j);
        cout<<res[j].first<<endl;
        cout<<res[j].second<<endl;
        printf("fin eigen %d \n",j);
    }



}

*/


In [ ]:
%%bash
#Esto es si queres ejecutar el segundo main del codigo de arriba, el cual hace el test de una matriz de Householder random con autovalores 1,..,150
# g++ -Ieigen eigen_types_test.cpp -o out
g++ -O3 eigen_types_test2.cpp -o out
./out

In [ ]:
%%bash
#esto es para compilar el primer main que escribe el output en archivos con el formato vi, λi, #iter, error (una linea por resultado)
g++ -O3 eigen_test2.cpp -o out

In [ ]:
def test_error():
  CANTIDAD_ITERACIONES = 100
  b = np.array([1, 0, 0, 0, 0], dtype=np.float64)

  #! Traten de hacer las cosas usando listas y nombres mas representativos
  epsilon_list = []
  error_listv1 = []
  error_listv2 = []
  error_listv3 = []
  error_listv4 = []
  error_listv5 = []

  iter_listv1 = []
  iter_listv2 = []
  iter_listv3 = []
  iter_listv4 = []
  iter_listv5 = []


  desv_stdv1fin = []
  desv_stdv2fin = []
  desv_stdv3fin = []
  desv_stdv4fin = []
  desv_stdv5fin = []

  # desv_stdv1 = []
  # desv_stdv2 = []
  # desv_stdv3 = []
  # desv_stdv4 = []
  # desv_stdv5 = []
  #pasos_list= []
  promv1 = 0
  promv2 = 0
  promv3 = 0
  promv4 = 0
  promv5 = 0

  promv1it = 0
  promv2it = 0
  promv3it = 0
  promv4it = 0
  promv5it = 0
  for epsilon in np.logspace(-4, 0, num=10):
    epsilon_list.append(epsilon)
    promv1 = 0
    promv2 = 0
    promv3 = 0
    promv4 = 0
    promv5 = 0
    promv1it = 0
    promv2it = 0
    promv3it = 0
    promv4it = 0
    promv5it = 0
    desv_stdv1fin = []
    desv_stdv2fin = []
    desv_stdv3fin = []
    desv_stdv4fin = []
    desv_stdv5fin = []

    for i in range(0, CANTIDAD_ITERACIONES):

      D = np.diag([10, 10-epsilon, 5, 2, 1])

      v = np.random.rand(D.shape[0])
      #print(v)

      v = v / np.linalg.norm(v)
      #print(v)

    # Matriz de Householder
      v = v.reshape(-1, 1)
      B = np.eye(D.shape[0]) - 2 * (v @ np.transpose(v))
      #print(v @ np.transpose(v))

    # Matriz a diagonalizar
      matriz_hh =  np.transpose(B) @ D @ B

      #print(matriz_hh)

      !rm input_data.txt
      with open('input_data.txt','a') as f:
          f.write(f"{matriz_hh.shape[0]} {matriz_hh.shape[1]}\n")
          np.savetxt(f, matriz_hh, newline="\n")
          np.savetxt(f ,b.reshape(1,-1), fmt='%1.3f', newline="\n")
      !./out input_data.txt f"output_data_{epsilon}".txt

      with open(f'foutput_data_{epsilon}.txt') as f:
        txt = str.split(f.read(), '\n')
        promv1 += float(txt[3])
        # promv2 += float(txt[7])
        # promv3 += float(txt[11])
        # promv4 += float(txt[15])
        # promv5 += float(txt[19])
        promv1it += float(txt[2])
        # promv2it += float(txt[6])
        # promv3it += float(txt[10])
        # promv4it += float(txt[14])
        # promv5it += float(txt[18])
        desv_stdv1fin.append(float(txt[3]))
        # desv_stdv2fin.append(float(txt[7]))
        # desv_stdv3fin.append(float(txt[11]))
        # desv_stdv4fin.append(float(txt[15]))
        # desv_stdv5fin.append(float(txt[19]))

    error_listv1.append(promv1/CANTIDAD_ITERACIONES)
    # error_listv2.append(promv2/CANTIDAD_ITERACIONES)
    # error_listv3.append(promv3/CANTIDAD_ITERACIONES)
    # error_listv4.append(promv4/CANTIDAD_ITERACIONES)
    # error_listv5.append(promv5/CANTIDAD_ITERACIONES)

    iter_listv1.append(promv1it/CANTIDAD_ITERACIONES)
    # iter_listv2.append(promv2it/CANTIDAD_ITERACIONES)
    # iter_listv3.append(promv3it/CANTIDAD_ITERACIONES)
    # iter_listv4.append(promv4it/CANTIDAD_ITERACIONES)
    # iter_listv5.append(promv5it/CANTIDAD_ITERACIONES)



  return [epsilon_list,error_listv1,iter_listv1,desv_stdv1fin]

data = test_error()


In [ ]:

std_errorv1 = np.std(data[3])
# std_errorv2 = np.std(data[12])
# std_errorv3 = np.std(data[13])
# std_errorv4 = np.std(data[14])
# std_errorv5 = np.std(data[15])

plt.yscale('log') # escala log que muestra...?
plt.xscale('log') # escala log que muestra los epsilons tomados espaciados logaritmicamente
plt.xlabel('ɛ')
plt.ylabel('||Avi-λvi||2')

plt.plot(data[0], data[1], '.k',label="v1",color="red")
plt.plot(data[0], data[1], '.k',label="v1",color="red")
#plt.fill_between(data[0], data[1] + std_errorv1, data[1] - std_errorv1, alpha = 0.2, color = 'red')

# plt.plot(data[0], data[2],'.k',label="v2",color="green")
# plt.fill_between(data[0], data[2] + std_errorv2, data[2] - std_errorv2, alpha = 0.2, color = 'green')


# plt.plot(data[0], data[3],'.k',label="v3",color="blue")
# plt.fill_between(data[0], data[3] + std_errorv3, data[3] - std_errorv3, alpha = 0.2, color = 'blue')

# plt.plot(data[0], data[4],'.k',label="v4",color="yellow")
# plt.fill_between(data[0], data[4] + std_errorv4, data[4] - std_errorv4, alpha = 0.2, color = 'yellow')


# plt.plot(data[0], data[5],'.k',label="v5",color="black")
# plt.fill_between(data[0], data[5] + std_errorv5, data[5] - std_errorv5, alpha = 0.2, color = 'black')

#plt.margins(x=0, y=10)
plt.legend(loc="upper right")

In [ ]:


plt.yscale('log') # escala log que muestra...?
plt.xscale('log') # escala log que muestra los epsilons tomados espaciados logaritmicamente
plt.xlabel('ɛ')
plt.ylabel('#iteraciones')

plt.plot(data[0], data[6], '.k',label="v1",color="red")
plt.plot(data[0], data[7],'.k',label="v2",color="green")
plt.plot(data[0], data[8],'.k',label="v3",color="blue")
plt.plot(data[0], data[9],'.k',label="v4",color="yellow")
plt.plot(data[0], data[10],'.k',label="v5",color="black")
plt.legend(loc="upper right")